### torch.index_select(input,dim,index) -> Tensor
- returns new tensor indexes the input tensor along dimension dim using entries in index, which is a longtensor
- 인덱스와 axis로 tensor value 선택하여 새로운 tensor 반환
- index : 1-D tensor

In [2]:
import torch
>>> x = torch.randn(3, 4)
>>> x
tensor([[ 0.1427,  0.0231, -0.5414, -1.0009],
        [-0.4664,  0.2647, -0.1228, -1.1068],
        [-1.1734, -0.6571,  0.7230, -0.6004]])
>>> indices = torch.tensor([0, 2])
>>> torch.index_select(x, 0, indices)
tensor([[ 0.1427,  0.0231, -0.5414, -1.0009],
        [-1.1734, -0.6571,  0.7230, -0.6004]])
>>> torch.index_select(x, 1, indices)
tensor([[ 0.1427, -0.5414],
        [-0.4664, -0.1228],
        [-1.1734,  0.7230]])

NameError: name 'tensor' is not defined

### torch.gather(input, dim, index)
- dim(axis)과 index 기준으로 input의 value 가져와서 새로운 tensor 생성
- index의 dim이 input의 dim과 같거나 작다. 즉 index_select 보다 더 입체적으로 value를 가져올 수 있다.


- out[i][j][k] = input[index[i][j][k]][j][k]  # if dim == 0<br>
out[i][j][k] = input[i][index[i][j][k]][k]  # if dim == 1<br>
out[i][j][k] = input[i][j][index[i][j][k]]  # if dim == 2

In [13]:
t = torch.tensor([[1, 2], [3, 4]])
indices0 = torch.tensor([[0,1],[1,1]])
indices1 = torch.tensor([[1,0],[0,1]])
t


tensor([[1, 2],
        [3, 4]])

#### dim = 1
- input index 둘 다 axis = 1 방향으로 

In [14]:
torch.gather(t, 1, torch.tensor([[0,1],[1,1]])

tensor([[1, 2],
        [4, 4]])

In [20]:
torch.gather(t, 1, torch.tensor([[1,0],[0,1]]))

tensor([[2, 1],
        [3, 4]])

#### 

#### dim = 0
- 혼란함
- input, index 둘 다 axis 0 방향으로 해석

In [23]:
torch.gather(t,0,torch.tensor([[0,1],[0,1]]))

tensor([[1, 4],
        [1, 4]])

In [19]:
torch.gather(t, 0, torch.tensor([[1,1],[0,0]]))

tensor([[3, 4],
        [1, 2]])

### tensor.scatter_(dim,src,index)
- index의 value를 target tensor의 index 삼아 src의 값을 넣어준다
    - dim의 방향대로 target과 src에 접근한다
    - target tensor의 index 생성 : <br>
   target tensor index의 dimension 'dim'에 index의 value, 이외 dimension에는 index value의 해당 dimension 위치 값을 할당하여 target tensor의 index를 생성한다.
        - dim = 0 , index = [[a,b]] then target[a,__0__], target[b,__1__], where index value a's dimension 위치 값 = (0,__0__) and index value b's dimension 위치 값 = (0,__1__)
        - dim = 1 , index = [[a],[b]] then target[0,a], target[1,b], where index value a's dimension 위치 값 = (__0__,0) and index value b's dimension 위치 값 = (__1__,0)
    - 생성된 target index에 src의 value를 넣는다. 이때 scatter 할 src의 value 또한 dim 방향으로 접근한다.
        - if src = [[1,2],[3,4]], and dim = 0 , then 1, 3, 2, 4 순서대로 target에 넣는다
    - self[index[i][j][k]][j][k] = src[i][j][k]  # if dim == 0<br>
self[i][index[i][j][k]][k] = src[i][j][k]  # if dim == 1<br>
self[i][j][index[i][j][k]] = src[i][j][k]  # if dim == 2
                    
- gather와 반대방향의 api

In [ ]:
>>> src = torch.arange(1, 11).reshape((2, 5))
>>> src
tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10]])
>>> index = torch.tensor([[0, 1, 2, 0]])
>>> torch.zeros(3, 5, dtype=src.dtype).scatter_(0, index, src)
tensor([[1, 0, 0, 4, 0],
        [0, 2, 0, 0, 0],
        [0, 0, 3, 0, 0]])
>>> index = torch.tensor([[0, 1, 2], [0, 1, 4]])
>>> torch.zeros(3, 5, dtype=src.dtype).scatter_(1, index, src)
tensor([[1, 2, 3, 0, 0],
        [6, 7, 0, 0, 8],
        [0, 0, 0, 0, 0]])

>>> torch.full((2, 4), 2.).scatter_(1, torch.tensor([[2], [3]]),
...            1.23, reduce='multiply')
tensor([[2.0000, 2.0000, 2.4600, 2.0000],
        [2.0000, 2.0000, 2.0000, 2.4600]])
>>> torch.full((2, 4), 2.).scatter_(1, torch.tensor([[2], [3]]),
...            1.23, reduce='add')
tensor([[2.0000, 2.0000, 3.2300, 2.0000],
        [2.0000, 2.0000, 2.0000, 3.2300]])

옵션 reduce='add',or'multipy'
- target tensor에 src값을 더거나 곱하여 값을 누적시킬 수 있다.

>>> torch.full((2, 4), 2.).scatter_(1, torch.tensor([[2], [3]]),
...            1.23, reduce='multiply')
tensor([[2.0000, 2.0000, 2.4600, 2.0000],
        [2.0000, 2.0000, 2.0000, 2.4600]])
>>> torch.full((2, 4), 2.).scatter_(1, torch.tensor([[2], [3]]),
...            1.23, reduce='add')
tensor([[2.0000, 2.0000, 3.2300, 2.0000],
        [2.0000, 2.0000, 2.0000, 3.2300]])